# Анализ результатов эксперимента от 30.09.25

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_df(ax, df, tags=None, title="Training curves"):
    for tag in tags:
        sns.lineplot(data=df, x="step", y=tag, ax=ax, label=tag)
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Step")
    ax.set_ylabel("Value")
    ax.legend(title="Metric")

## Описание эксперимента

Обучение нейронной сети с учетом всех нароботок, включая предварительный сбор данных.

## Результаты

In [ ]:
from pathlib import Path

EXPERIMENT_NAME = "td3_train_real_async"
EXPERIMENT_DATE = "2025-09-30"
EXPERIMENT_TIME = "13-08-38"

PATH_TO_EXP_DIR = Path(f"../experiments/{EXPERIMENT_NAME}/{EXPERIMENT_DATE}_{EXPERIMENT_TIME}")

ENV_LOG_DIR = PATH_TO_EXP_DIR / "env_logs"
TRAIN_LOG_DIR = PATH_TO_EXP_DIR / "train_logs"

In [ ]:
import re
import pandas as pd

log_pattern_interaction = re.compile(
    r"step=(?P<step>\d+)"
    r" time=(?P<time>\d+\.\d+)"
    r" phase=(?P<phase>\w+)"
    r" block_step=(?P<block_step>\d+)"
    r" kp=(?P<kp>-?\d+\.\d+)"
    r" ki=(?P<ki>-?\d+\.\d+)"
    r" kd=(?P<kd>-?\d+\.\d+)"
    r" process_variable=(?P<process_variable>-?\d+\.\d+)"
    r" control_output=(?P<control_output>-?\d+\.\d+)"
    r" u_min=(?P<u_min>-?\d+\.\d+)"
    r" u_max=(?P<u_max>-?\d+\.\d+)"
)

log_pattern_model = re.compile(
    r"step=(?P<step>\d+)"
    r" phase=(?P<phase>\w+)"
    r" block_step=(?P<block_step>final)"
    r" kp=(?P<kp>-?\d+\.\d+)"
    r" ki=(?P<ki>-?\d+\.\d+)"
    r" kd=(?P<kd>-?\d+\.\d+)"
    r" error_mean=(?P<error_mean>-?\d+\.\d+)"
    r" error_std=(?P<error_std>-?\d+\.\d+)"
    r" error_mean_norm=(?P<error_mean_norm>-?\d+\.\d+)"
    r" error_std_norm=(?P<error_std_norm>-?\d+\.\d+)"
    r" reward=(?P<reward>-?\d+\.\d+)"
)

def parse_logs_split(file_path: str):
    interaction, model = [], []
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("step"):
                m1 = log_pattern_interaction.match(line)
                if m1:
                    interaction.append(m1.groupdict())
                    continue
                m2 = log_pattern_model.match(line)
                if m2:
                    model.append(m2.groupdict())
                    continue
    interaction_df = pd.DataFrame(interaction)
    model_df = pd.DataFrame(model)

    for df in (interaction_df, model_df):
        if not df.empty:
            df["step"] = df["step"].astype(int)
            for col in df.columns:
                if col != "phase" and col != "block_step":
                    df[col] = pd.to_numeric(df[col], errors="coerce")

    return interaction_df, model_df


interaction_df, model_df = parse_logs_split(ENV_LOG_DIR / "log.txt")

In [ ]:
print("=== Interaction logs info ===")
print(interaction_df.info())
print("\n=== Interaction logs head ===")
print(interaction_df.head())
print("\n=== Interaction logs describe ===")
print(interaction_df.describe())

## Графики

In [ ]:
filtered_interaction_df = interaction_df[interaction_df["phase"] == "normal"].copy()

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

sns.lineplot(data=filtered_interaction_df, x="step", y="process_variable", ax=axes[0], color="blue")
axes[0].set_title("Process Variable")
axes[0].set_ylabel("process_variable")

sns.lineplot(data=filtered_interaction_df, x="step", y="control_output", ax=axes[1], color="red")
axes[1].set_title("Control Output")
axes[1].set_xlabel("Step")
axes[1].set_ylabel("control_output")

plt.tight_layout()
plt.show()

In [ ]:
print("=== Model logs info ===")
print(model_df.info())
print("\n=== Model logs head ===")
print(model_df.head())
print("\n=== Model logs describe ===")
print(model_df.describe())

Фильтрация по фазе тренировки.

In [ ]:
NUM_STEPS_IN_BLOCK = 200
from nn_laser_stabilizer.envs.pid_tuning_experimental_env import Phase

filtered_df = model_df[model_df["phase"] == Phase.NORMAL].copy()
filtered_df["block"] = filtered_df["step"] / NUM_STEPS_IN_BLOCK

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

sns.lineplot(data=filtered_df, x="block", y="kp", ax=axes[0], color="blue")
axes[0].set_title("kp over Steps")
axes[0].set_ylabel("kp")

sns.lineplot(data=filtered_df, x="block", y="ki", ax=axes[1], color="orange")
axes[1].set_title("ki over Steps")
axes[1].set_ylabel("ki")

sns.lineplot(data=filtered_df, x="block", y="kd", ax=axes[2], color="green")
axes[2].set_title("kd over Steps")
axes[2].set_xlabel("Block Step")
axes[2].set_ylabel("kd")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

sns.lineplot(data=filtered_df, x="block", y="error_mean", ax=axes[0], color="blue")
axes[0].set_title("Error Mean (Raw)")
axes[0].set_ylabel("error_mean")

sns.lineplot(data=filtered_df, x="block", y="error_mean_norm", ax=axes[1], color="red")
axes[1].set_title("Error Mean (Normalized)")
axes[1].set_xlabel("Block Step")
axes[1].set_ylabel("error_mean_norm")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

sns.lineplot(data=filtered_df, x="block", y="error_std", ax=axes[0], color="blue")
axes[0].set_title("Error STD (Raw)")
axes[0].set_ylabel("error_std")

sns.lineplot(data=filtered_df, x="block", y="error_std_norm", ax=axes[1], color="red")
axes[1].set_title("Error STD (Normalized)")
axes[1].set_xlabel("Block Step")
axes[1].set_ylabel("error_std_norm")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(data=filtered_df, x="block", y="reward", ax=ax, color="green")
ax.set_title("Reward over Steps")
ax.set_xlabel("Block Step")
ax.set_ylabel("Reward")
plt.show()

## Статистика

In [ ]:
filtered_model_df = model_df[model_df["phase"] == "pretrain"].copy()

print("=== Model logs info (phase == 'pretrain') ===")
print(filtered_model_df.info())
print("\n=== Model logs head (phase == 'pretrain') ===")
print(filtered_model_df.head())
print("\n=== Model logs describe (phase == 'pretrain') ===")
print(filtered_model_df.describe())

Новые коэффициенты для нормализации будут 250 для mean и 200 для std. 

## Ход обучения

In [ ]:
import re
import pandas as pd

log_pattern_rl = re.compile(
    r"step=(?P<step>\d+)\s+Loss/Critic=(?P<critic_loss>-?\d+\.\d+)"
    r"\s+Loss/Actor=(?P<actor_loss>-?\d+\.\d+)"
)

def parse_rl_logs(file_path: str):
    rows = []
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            m = log_pattern_rl.match(line)
            if m:
                rows.append({
                    "step": int(m.group("step")),
                    "critic_loss": float(m.group("critic_loss")),
                    "actor_loss": float(m.group("actor_loss")),
                })
    df = pd.DataFrame(rows)
    return df

rl_logs_df = parse_rl_logs(TRAIN_LOG_DIR / "log.txt") 

In [ ]:
print("RL training logs info:")
print(rl_logs_df.info())
print("\nRL training logs head:")
print(rl_logs_df.head())
print("\nRL training logs describe:")
print(rl_logs_df.describe())

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
plot_df(ax, rl_logs_df, tags=["critic_loss", "actor_loss"], title="Training loss")